In [ ]:
pip install ultralytics

In [5]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
print("YOLOv8 loaded successfully")

YOLOv8 loaded successfully


In [6]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

CUDA available: True
GPU: Tesla T4
